In [1]:
import pandas as pd
import numpy as np

In [2]:
temp_ari = pd.read_csv("open-meteo-50.86N4.33E26m.csv")
temp_ari_hum = pd.read_csv("open-meteo-50.86N4.33E26m_hum.csv")
temp_ill = pd.read_csv("open-meteo-48.19N16.38E179m_ili.csv")
temp_ill_hum = pd.read_csv("open-meteo-48.19N16.38E179m_hum.csv")                           

C:\Users\flore\AppData\Local\Temp\ipykernel_20928\2741939497.py:2: DtypeWarning: Columns (0,2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_ari_hum = pd.read_csv("open-meteo-50.86N4.33E26m_hum.csv")
C:\Users\flore\AppData\Local\Temp\ipykernel_20928\2741939497.py:4: DtypeWarning: Columns (0,2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_ill_hum = pd.read_csv("open-meteo-48.19N16.38E179m_hum.csv")


In [17]:
def function_get_data(df,n_countries,n_columns):
    data_description = df.iloc[0:n_countries+1,]
    aux_column_names = list(df.iloc[n_countries+1,0:n_columns+1])
    temp_data = df.iloc[n_countries+2:,0:n_columns+1]
    temp_data.columns = aux_column_names
    return data_description,temp_data

description_ari,data_ari = function_get_data(temp_ari,12,3)
description_ari_hum,data_ari_hum = function_get_data(temp_ari_hum,12,2)
description_ill,data_ill = function_get_data(temp_ill,18,3)
description_ill_hum,data_ill_hum = function_get_data(temp_ill_hum,18,2)

In [35]:
data_ari['DateTime'] = pd.to_datetime(data_ari['time'])
data_ari['temperature_2m_max (°C)'] = pd.to_numeric(data_ari['temperature_2m_max (°C)'])
data_ari['temperature_2m_min (°C)'] = pd.to_numeric(data_ari['temperature_2m_min (°C)'])
data_ari_hum['DateTime'] = pd.to_datetime(data_ari_hum['time'])
data_ari_hum['relative_humidity_2m (%)'] = pd.to_numeric(data_ari_hum['relative_humidity_2m (%)'])
data_ill['DateTime'] = pd.to_datetime(data_ill['time'])
data_ill['temperature_2m_max (°C)'] = pd.to_numeric(data_ill['temperature_2m_max (°C)'])
data_ill['temperature_2m_min (°C)'] = pd.to_numeric(data_ari_hum['relative_humidity_2m (%)'])
data_ill_hum['DateTime'] = pd.to_datetime(data_ill_hum['time'])
data_ill_hum['relative_humidity_2m (%)'] = pd.to_numeric(data_ill_hum['relative_humidity_2m (%)'])


In [40]:
data_ill_hum['Date'] = data_ill_hum['DateTime'].dt.strftime('%Y-%m-%d')
data_ari_hum['Date'] = data_ari_hum['DateTime'].dt.strftime('%Y-%m-%d')

In [ ]:
data_ill_hum_by_day = data_ill_hum.groupby(['location_id', 'Date'])['relative_humidity_2m (%)'].mean().reset_index()
data_ari_hum_by_day = data_ari_hum.groupby(['location_id','Date'])['relative_humidity_2m (%)'].mean().reset_index()


,location_id,Date,relative_humidity_2m (%)
0,1,2019-08-25,49.166667
1,1,2019-08-26,51.583333
2,1,2019-08-27,46.583333
3,1,2019-08-28,45.875000
4,1,2019-08-29,42.375000
...,...,...,...
39946,1,2019-08-21,48.875000
39947,1,2019-08-22,48.958333
39948,1,2019-08-23,49.333333
39949,1,2019-08-24,54.416667


In [57]:
print(data_ill_hum_by_day.sort_values(by=['Date','location_id']).head())
print(data_ill_hum_by_day.tail())
print(data_ill_hum_by_day['Date'].nunique())

      location_id        Date  relative_humidity_2m (%)
1874            2  2014-09-22                 73.333333
5541            3  2014-09-22                 73.958333
9208            4  2014-09-22                 93.041667
12869           5  2014-09-22                 72.958333
16536           6  2014-09-22                 63.750000
      location_id        Date  relative_humidity_2m (%)
69119           1  2019-08-16                 72.625000
69120           1  2019-08-17                 82.625000
69121           1  2019-08-18                 80.083333
69122           1  2019-08-19                 69.958333
69123           1  2019-08-20                 82.916667
3667


In [51]:
print(data_ari_hum_by_day.tail())
print(data_ari.tail())

      location_id        Date  relative_humidity_2m (%)
39946           1  2019-08-21                 48.875000
39947           1  2019-08-22                 48.958333
39948           1  2019-08-23                 49.333333
39949           1  2019-08-24                 54.416667
39950           1  2019-08-25                 51.722222
      location_id        time  temperature_2m_max (°C)  \
39959          12  2024-09-26                     20.8   
39960          12  2024-09-27                     22.1   
39961          12  2024-09-28                     16.7   
39962          12  2024-09-29                     15.7   
39963          12  2024-09-30                     15.7   

       temperature_2m_min (°C)   DateTime  
39959                     13.6 2024-09-26  
39960                     15.9 2024-09-27  
39961                     10.9 2024-09-28  
39962                      6.7 2024-09-29  
39963                      4.7 2024-09-30  
